In [15]:
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import numpy as np
import random

In [ ]:
import gymnasium as gym

In [16]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [14]:
env = ShowerEnv()

C:\Users\donvi\.conda\envs\DROO\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [15]:
env.observation_space.sample()

array([79.04312], dtype=float32)

In [17]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-56
Episode:2 Score:-28
Episode:3 Score:-28
Episode:4 Score:-44
Episode:5 Score:-50
Episode:6 Score:-44
Episode:7 Score:16
Episode:8 Score:-30
Episode:9 Score:2
Episode:10 Score:-56


In [17]:
from ray.rllib.algorithms import ppo
import ray
import argparse

In [4]:
ray.init()

2023-04-30 22:12:51,621	INFO worker.py:1622 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.7.15
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8265


In [5]:
parser = argparse.ArgumentParser()
parser.add_argument(
    "--run", type=str, default="PPO", help="The RLlib-registered algorithm to use."
)
parser.add_argument(
    "--framework",
    choices=["tf", "tf2", "torch"],
    default="tf2",
    help="The DL framework specifier.",
)
parser.add_argument(
    "--as-test",
    action="store_true",
    help="Whether this script should be run as a test: --stop-reward must "
    "be achieved within --stop-timesteps AND --stop-iters.",
)
parser.add_argument(
    "--stop-iters", type=int, default=50, help="Number of iterations to train."
)
parser.add_argument(
    "--stop-timesteps", type=int, default=100000, help="Number of timesteps to train."
)
parser.add_argument(
    "--stop-reward", type=float, default=0.1, help="Reward at which we stop training."
)
parser.add_argument(
    "--no-tune",
    action="store_true",
    help="Run without Tune using a manual train loop instead. In this case,"
    "use PPO without grid search and no TensorBoard.",
)
parser.add_argument(
    "--local-mode",
    action="store_true",
    help="Init Ray in local mode for easier debugging.",
)


_StoreTrueAction(option_strings=['--local-mode'], dest='local_mode', nargs=0, const=True, default=False, type=None, choices=None, help='Init Ray in local mode for easier debugging.', metavar=None)

In [6]:
algo = dqn.DQN(env=ShowerEnv, config ={"env_config": {},})

2023-04-30 22:13:01,794	WARNING deprecation.py:51 -- DeprecationWarning: `algo = Algorithm(env='<class '__main__.ShowerEnv'>', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('<class '__main__.ShowerEnv'>').build()` instead. This will raise an error in the future!
2023-04-30 22:13:01,846	INFO algorithm.py:528 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


TypeError: __init__() takes 1 positional argument but 2 were given

In [46]:
!pip install torch

     ------------------------------------- 162.6/162.6 MB 13.6 MB/s eta 0:00:00


In [9]:
class MyEnv(Env):
    def __init__(self, env_config):
        self.action_space = Discrete(3)
        self.observation_space = Discrete(3)
    def reset(self):
        return self.observation_space
    def step(self, action):
        reward = 5
        done = 1
        
        return self.observation_space, reward, done, {}

In [18]:
algo = ppo.PPO(env=MyEnv, config={
    "env_config": {},  # config to pass to env class
})

2023-04-30 22:17:07,730	WARNING deprecation.py:51 -- DeprecationWarning: `algo = Algorithm(env='<class '__main__.MyEnv'>', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('<class '__main__.MyEnv'>').build()` instead. This will raise an error in the future!
(pid=13716) WARNING:tensorflow:From C:\Users\donvi\.conda\envs\DROO\lib\site-packages\tensorflow\python\util\deprecation.py:561: calling function (from tensorflow.python.eager.def_function) with experimental_relax_shapes is deprecated and will be removed in a future version. [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(pid=13716) Instructions for updating: [repeated 2x across cluster]
(pid=13716) experimental_relax_shapes is deprecated, use reduce_retracing instead [repeated 2x across cluster]
(RolloutWorker pid=24420) experimental_rela

2023-04-30 22:17:13,362	ERROR actor_manager.py:508 -- Ray error, taking actor 1 out of service. The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=6440, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x0000018A22D1F3C8>)
  File "C:\Users\donvi\.conda\envs\DROO\lib\site-packages\ray\rllib\utils\pre_checks\env.py", line 138, in check_gym_environments
    raise ValueError(ERR_MSG_OLD_GYM_API.format(env, ""))
ValueError: Your environment (<MyEnv instance>) does not abide to the new gymnasium-style API!
From Ray 2.3 on, RLlib only supports the new (gym>=0.26 or gymnasium) Env APIs.

Learn more about the most important changes here:
https://github.com/openai/gym and here: https://github.com/Farama-Foundation/Gymnasium

In order to fix this problem, do the following:

1) Run `pip install gymnasium` on your command line.
2) Change all your import statements in your code from
   `import gym` -> `import gymnasium as

ValueError: Traceback (most recent call last):
  File "C:\Users\donvi\.conda\envs\DROO\lib\site-packages\ray\rllib\utils\pre_checks\env.py", line 81, in check_env
    check_gym_environments(env)
  File "C:\Users\donvi\.conda\envs\DROO\lib\site-packages\ray\rllib\utils\pre_checks\env.py", line 138, in check_gym_environments
    raise ValueError(ERR_MSG_OLD_GYM_API.format(env, ""))
ValueError: Your environment (<MyEnv instance>) does not abide to the new gymnasium-style API!
From Ray 2.3 on, RLlib only supports the new (gym>=0.26 or gymnasium) Env APIs.

Learn more about the most important changes here:
https://github.com/openai/gym and here: https://github.com/Farama-Foundation/Gymnasium

In order to fix this problem, do the following:

1) Run `pip install gymnasium` on your command line.
2) Change all your import statements in your code from
   `import gym` -> `import gymnasium as gym` OR
   `from gym.space import Discrete` -> `from gymnasium.spaces import Discrete`

For your custom (s

(RolloutWorker pid=13716) 2023-04-30 22:17:13,355	ERROR worker.py:844 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=13716, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x000002698784B7C8>)
(RolloutWorker pid=13716) During handling of the above exception, another exception occurred:
(RolloutWorker pid=13716) ray::RolloutWorker.__init__() (pid=13716, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x000002698784B7C8>)
(RolloutWorker pid=13716)   File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor
(RolloutWorker pid=13716)   File "C:\Users\donvi\.conda\envs\DROO\lib\site-packages\ray\_private\function_manager.py", line 670, in actor_method_executor
(RolloutWorker pid=13716)     return method(__ray_actor, *args, **kwargs)
(RolloutWorker pid=13716)   File "C:\Users\donvi\.conda\envs\DROO\lib\site-pack